In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import cm
import matplotlib.pyplot as plt

# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python3 --version
import metnum

/home/john/Desktop/TP/tp3-metnum/notebooks
Python 3.8.3


In [2]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.feature_selection import  SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def errorRMSE(y1,y2 ):
    return sqrt ( mean_squared_error(y1,y2) )

def errorRMSLE(y1,y2 ):
    return sqrt ( mean_squared_log_error(np.abs(y1),np.abs(y2)) )

In [3]:
df = pd.read_csv('../data/train.csv')
df['fecha']  = df['fecha'].apply(lambda x:   (int( x.split('-')[0]) + int(x.split('-')[1] )/13.0 )  )
# df['es_nuevo']  = df['antiguedad'].apply(lambda x: 1 if x <=1 else 0  )
df['descripcion']  = df['descripcion'].apply(lambda x:   ( len ( str ( x))) )
df= df.loc[:, df.columns != 'titulo']
# df= df.loc[:, df.columns != 'antiguedad']
df= df.loc[:, df.columns != 'tipodepropiedad']
df= df.loc[:, df.columns != 'direccion']
df= df.loc[:, df.columns != 'ciudad']
df= df.loc[:, df.columns != 'provincia']
# df= df.loc[:, df.columns != 'lng']
# df= df.loc[:, df.columns != 'lat']
# df= df.loc[:, df.columns != 'id']
# df= df.loc[:, df.columns != 'idzona']
df.head() 

,id,descripcion,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,171,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,NaN,NaN,2015.615385,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,716,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,19.310205,-99.227655,2013.461538,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,1340,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,NaN,NaN,2015.769231,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,357,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,19.301890,-99.688015,2012.230769,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,103,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,NaN,NaN,2016.461538,0.0,0.0,0.0,0.0,0.0,1150000.0


In [4]:
maxcolumnas = len(df.columns) 
df = df.dropna()
cols = list(df)
cols.insert(0, cols.pop(cols.index('precio')))
df = df.loc[:, cols]
ix = df.corr().abs().sort_values('precio', ascending=False).index
df = df.loc[:, ix]
df.head()
for col in df.columns: 
    print(col) 

precio
metroscubiertos
banos
metrostotales
garages
gimnasio
habitaciones
lat
fecha
usosmultiples
piscina
idzona
antiguedad
lng
centroscomercialescercanos
descripcion
escuelascercanas
id


In [7]:
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.feature_selection import  SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def errorRMSE(y1,y2 ):
    return sqrt ( mean_squared_error(y1,y2) )

def errorRMSLE(y1,y2 ):
    return sqrt ( mean_squared_log_error( (y1), (y2)) )

for i in range(2,maxcolumnas+1):
#     print(i-1)
    dfLocal =  df.iloc[:,1:i].copy()
#     print(dfLocal)
    dfTrain = dfLocal.loc[:, dfLocal.columns != 'precio']

    X_train = dfTrain.values
    n =len(dfTrain.columns)

    y_train = df['precio'].values
    precio_minimo =y_train.min()
    X = X_train.reshape(len(X_train), n)
    y = y_train.reshape(len(y_train),)

    from sklearn.model_selection import KFold
    def aplicarCota(lista, cota):
        for i, valor in enumerate(lista):
            if valor < cota:
                lista[i] = cota
    #     return lista

    eRMSE  = 0
    eRLMSE  = 0
    splits = 5
    kf = KFold(n_splits=splits, random_state=42, shuffle=True)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        lr = metnum.LinearRegression()

        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)
        aplicarCota(y_pred, precio_minimo)
        eRMSE += errorRMSE(y_test,y_pred) 
        eRLMSE += errorRMSLE(y_test,y_pred)  
    eRMSE = (eRMSE/(splits) )
    eRLMSE = (eRLMSE/splits)
    print(i-1)
    print(eRMSE)
    print(eRLMSE)

1
1421433.9018644833
0.5763563554449427
2
1399533.2233187293
0.5848537303180269
3
1393911.8623422205
0.592931799377223
4
1388935.8329770726
0.6016558595007149
5
1352956.537234346
0.584268878407764
6
1312775.8516876716
0.550057315847535
7
1307718.08052307
0.5405288111596915
8
1277124.5637075645
0.5224075163586792
9
1275473.5186809346
0.5210711351116957
10
1275486.3911888844
0.5210919525775793
11
1259902.6741532243
0.5121815018681599
12
1251980.0101682672
0.50862763490546
13
1244371.9872049273
0.5071836921368462
14
1243712.5119111196
0.5067365909313641
15
1243303.0525660133
0.5066794785061136
16
1239749.7597950066
0.50597138936008
17
1239707.5262524826
0.50603685654981


In [6]:
#habitaciones:
